# **Import Data Library**

In [10]:
import pandas as pd                         # Import Pandas
import statsmodels.api as sm                                  
data = pd.read_csv("cleanDiamods.csv")      # Import dataset 

# **Split Train and Test**

In [11]:
train = data.head(int(len(data)*0.7))
test = data.tail(int(len(data)*0.3))

In [12]:
y = train["price"]
x = train[["carat","x","y","z","table","depth","cut","color","clarity"]]
x = sm.add_constant(x)

In [13]:
mod = sm.OLS(y, x)                          # Create Modal
results = mod.fit()                         # Fit Modal
results.summary()                 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     9709.
Date:                Tue, 09 Nov 2021   Prob (F-statistic):               0.00
Time:                        22:28:40   Log-Likelihood:                -92225.
No. Observations:               15880   AIC:                         1.845e+05
Df Residuals:                   15870   BIC:                         1.845e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1729.7852    450.066     -3.843      0.000   -2611.966    -847.605
carat       2968.0745     97.511     30.438      0.000    2776.941    3159.208
x           1902.4744     51.585     36.881      0.000    1801.363    2003.586
y          -1426.2166     51.419    -27.737      0.000   -1527.003   -1325.430
z           -800.5015    156.270     -5.123      0.000   -1106.809    -494.194
table         -0.6771      0.440     -1.538      0.124      -1.540       0.186
depth         33.5830      7.123      4.715      0.000      19.621      47.545
cut          -18.0011      0.815    -22.075      0.000     -19.599     -16.403
color        -48.1801      0.416   -115.721      0.000     -48.996     -47.364
clarity      -77.2035      0.426   -181.434      0.000     -78.038     -76.369
==============================================================================
Omnibus:                      359.464   Durbin-Watson:                   0.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              765.666
Skew:                           0.101   Prob(JB):                    5.47e-167
Kurtosis:                       4.057   Cond. No.                     6.33e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.33e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
price_predect = []
for i in range(test.index[0], test.index[-1] + 1):
    price_predect.append(-1729.7852
                    +2968.0745 *test["carat"][i] 
                    +1902.4744 *test["x"][i]
                    -1426.2166 *test["y"][i] 
                    -800.5015 *test["z"][i]
                    -0.6771	 *test["table"][i] 
                    +33.5830	 *test["depth"][i]
                    -18.0011 *test["cut"][i]
                    -48.1801 *test["color"][i]
                    -77.2035 *test["clarity"][i])

indexs = [i for i in range(test.index[-1] + 1 - test.index[0])]
d = dict(zip(indexs,price_predect) )


In [15]:
predict_data = pd.concat([test.reset_index() , pd.Series(d, name='predict')], axis = 'columns')
predict_data

,index,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z,predict
0,15881,41537,0.42,0,0,4,62.1,56.0,1235,4.82,4.78,2.98,1222.695580
1,15882,41538,0.42,0,0,4,61.5,54.0,1235,4.84,4.79,2.96,1243.697332
2,15883,41539,0.42,1,0,4,60.8,59.0,1235,4.85,4.79,2.93,1241.842421
3,15884,41540,0.42,0,0,4,61.1,57.0,1235,4.84,4.81,2.95,1207.713515
4,15885,41541,0.42,1,0,4,60.8,60.0,1235,4.87,4.84,2.95,1191.893949
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6801,22682,53635,0.30,0,5,2,62.1,56.0,552,4.30,4.33,2.68,672.694372
6802,22683,53636,0.30,0,0,6,61.6,57.0,552,4.28,4.32,2.65,587.539995
6803,22684,53637,0.30,2,0,6,62.5,57.0,552,4.26,4.28,2.67,584.751641
6804,22685,53638,0.30,0,0,6,61.0,57.0,552,4.29,4.33,2.63,588.162803


In [16]:
ans = []
for i in predict_data.index:
    d = predict_data["price"][i] - predict_data["predict"][i]
    ans.append(abs(d))
print("abs meen error =",sum(ans) / len(ans))

abs meen error = 268.0613693113422


In [17]:
import math
summm=0
for i in predict_data.index:
    d = predict_data["price"][i] - predict_data["predict"][i]
    summm+=d**2

rmsee=math.sqrt(summm/len(predict_data))
print("rmsee =",rmsee)

rmsee = 320.11959162225605


In [18]:
summm=0
for i in predict_data.index:
    d=predict_data["price"][i]-predict_data["predict"][i]
    summm+=abs(d)/predict_data["price"][i]

mape=(summm/len(predict_data))*100
print("mape =",mape)

mape = 17.291432243792567
